In [0]:
%pip install yfinance

In [0]:
from pyspark.sql.functions import col, when, concat, lit

angel_isin = spark.table("personal_finance.bronze.angel_one_holding_statement").select("ISIN")
groww_isin = spark.table("personal_finance.bronze.groww_holding_statement").select("ISIN")
kite_isin = spark.table("personal_finance.bronze.kite_holding_statement").select("ISIN")

consolidated_isin = angel_isin.union(groww_isin).union(kite_isin).distinct()

company_listing_info = spark.table("personal_finance.silver.company_listing_info")

result_df = (
    consolidated_isin
    .join(company_listing_info, col("ISINNumber") == col("ISIN"))
    .select(
        col("ISIN"),
        when(
            col("Securitytype") == "INVIT",
            concat(col("Symbol"), lit("-"), col("Series"))
        ).otherwise(col("Symbol")).alias("Symbol")
    )
    .distinct()
)

In [0]:
import yfinance as yf
import pandas as pd

symbols = [row['Symbol'] for row in result_df.select('Symbol').distinct().collect()]

info_list = []
for symbol in symbols:
    ticker = yf.Ticker(symbol + '.NS')
    info = ticker.info
    info_filtered = {
        k: str(v) if v is not None else None
        for k, v in info.items()
        if k != "companyOfficers"
    }
    info_list.append(info_filtered)

info_df = pd.DataFrame(info_list)

In [0]:
spark_df = spark.createDataFrame(info_df)
spark_df.write.mode("overwrite").saveAsTable("personal_finance.bronze.yfinance_company_info")

In [0]:
%sql
select * from personal_finance.bronze.yfinance_company_info